# 📓 🦜️🔗 _LangChain_ Integration

TruLens provides TruChain, a deep integration with _LangChain_ to allow you to
inspect and evaluate the internals of your application built using _LangChain_.
This is done through the instrumentation of key _LangChain_ classes. To see a list
of classes instrumented, see *Appendix: Instrumented _LangChain_ Classes and
Methods*.

In addition to the default instrumentation, TruChain exposes the
*select_context* method for evaluations that require access to retrieved
context. Exposing *select_context* bypasses the need to know the json structure
of your app ahead of time, and makes your evaluations re-usable across different
apps.

## Example Usage

Below is a quick example of usage. First, we'll create a standard LLMChain.

In [2]:
# required imports
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate
from trulens_eval import TruChain

# typical LangChain rag setup
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template=
        "Provide a helpful response with relevant background information for the following: {prompt}",
        input_variables=["prompt"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

llm = OpenAI(temperature=0.9, max_tokens=128)
chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

To instrument an LLM chain, all that's required is to wrap it using TruChain.

In [3]:
# instrument with TruChain
tru_recorder = TruChain(chain)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


Similarly, LangChain apps defined with LangChain Expression Language (LCEL) are also supported.

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

tru_recorder = TruChain(
    chain,
    app_id='Chain1_ChatApplication'
)

To properly evaluate LLM apps we often need to point our evaluation at an
internal step of our application, such as the retreived context. Doing so allows
us to evaluate for metrics including context relevance and groundedness.

For LangChain applications where the BaseRetriever is used, `select_context` can
be used to access the retrieved text for evaluation.

Example usage:

```python
context = TruChain.select_context(rag_chain)

f_context_relevance = (
    Feedback(provider.qs_relevance)
    .on_input()
    .on(context)
    .aggregate(np.mean)
    )
```

For added flexibility, the select_context method is also made available through
`trulens_eval.app.App`. This allows you to switch between frameworks without
changing your context selector:

```python
from trulens_eval.app import App
context = App.select_context(rag_chain)
```

You can find the full quickstart available here: [LangChain Quickstart](../langchain_quickstart)

## Async Support

TruChain also provides async support for _LangChain_ through the `acall` method. This allows you to track and evaluate async and streaming _LangChain_ applications.

As an example, below is an LLM chain set up with an async callback.

In [5]:
from langchain import LLMChain
from langchain.callbacks import AsyncIteratorCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from trulens_eval import TruChain

# Set up an async callback.
callback = AsyncIteratorCallbackHandler()

# Setup a simple question/answer chain with streaming ChatOpenAI.
prompt = PromptTemplate.from_template("Honestly answer this question: {question}.")
llm = ChatOpenAI(
    temperature=0.0,
    streaming=True, # important
    callbacks=[callback]
)
async_chain = LLMChain(llm=llm, prompt=prompt)

Once you have created the async LLM chain you can instrument it just as before.

In [6]:
async_tc_recorder = TruChain(async_chain)

with async_tc_recorder as recording:
    await async_chain.ainvoke(input=dict(question="What is 1+2? Explain your answer."))

For more usage examples, check out the [LangChain examples directory](https://github.com/truera/trulens/tree/main/trulens_eval/examples/expositional/frameworks/langchain).

## Appendix: Instrumented LangChain Classes and Methods

The modules, classes, and methods that trulens instruments can be retrieved from
the appropriate Instrument subclass.

In [7]:
from trulens_eval.tru_chain import LangChainInstrument
LangChainInstrument().print_instrumentation()

Module langchain*
  Class langchain.agents.agent.BaseMultiActionAgent
    Method plan: (self, intermediate_steps: 'List[Tuple[AgentAction, str]]', callbacks: 'Callbacks' = None, **kwargs: 'Any') -> 'Union[List[AgentAction], AgentFinish]'
    Method aplan: (self, intermediate_steps: 'List[Tuple[AgentAction, str]]', callbacks: 'Callbacks' = None, **kwargs: 'Any') -> 'Union[List[AgentAction], AgentFinish]'
  Class langchain.agents.agent.BaseSingleActionAgent
    Method plan: (self, intermediate_steps: 'List[Tuple[AgentAction, str]]', callbacks: 'Callbacks' = None, **kwargs: 'Any') -> 'Union[AgentAction, AgentFinish]'
    Method aplan: (self, intermediate_steps: 'List[Tuple[AgentAction, str]]', callbacks: 'Callbacks' = None, **kwargs: 'Any') -> 'Union[AgentAction, AgentFinish]'
  Class langchain.chains.base.Chain
    Method __call__: (self, inputs: Union[Dict[str, Any], Any], return_only_outputs: bool = False, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langch

### Instrumenting other classes/methods.
Additional classes and methods can be instrumented by use of the
`trulens_eval.instruments.Instrument` methods and decorators. Examples of
such usage can be found in the custom app used in the `custom_example.ipynb`
notebook which can be found in
`trulens_eval/examples/expositional/end2end_apps/custom_app/custom_app.py`. More
information about these decorators can be found in the
`docs/trulens_eval/tracking/instrumentation/index.ipynb` notebook.

### Inspecting instrumentation
The specific objects (of the above classes) and methods instrumented for a
particular app can be inspected using the `App.print_instrumented` as
exemplified in the next cell. Unlike `Instrument.print_instrumentation`, this
function only shows what in an app was actually instrumented.

In [8]:
async_tc_recorder.print_instrumented()

Components:
	TruChain (Other) at 0x2b60a3660 with path __app__
	LLMChain (Other) at 0x2b5cdb3e0 with path __app__.app
	PromptTemplate (Custom) at 0x2b605e580 with path __app__.app.prompt
	ChatOpenAI (Custom) at 0x2b5cdb4d0 with path __app__.app.llm
	StrOutputParser (Custom) at 0x2b60a3750 with path __app__.app.output_parser

Methods:
Object at 0x2b5cdb3e0:
	<function Chain.__call__ at 0x2a6c17560> with path __app__.app
	<function Chain.invoke at 0x2a6c16de0> with path __app__.app
	<function Chain.ainvoke at 0x2a6c16e80> with path __app__.app
	<function Chain.run at 0x2a6c17b00> with path __app__.app
	<function Chain.arun at 0x2a6c17d80> with path __app__.app
	<function LLMChain._call at 0x2a6c6c2c0> with path __app__.app
	<function LLMChain._acall at 0x2a6c6c860> with path __app__.app
	<function Chain.acall at 0x2a6c177e0> with path __app__.app
	<function Chain._call at 0x2a6c17380> with path __app__.app
	<function Chain._acall at 0x2a6c17420> with path __app__.app
	<function Runnable.